# **Worksapce set up**

In [24]:
# Connecting to google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [25]:
# Import libraries
%pylab inline
!pip install -q tensorflow-model-optimization

import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow.keras.optimizers import SGD
import numpy as np
import tempfile
import zipfile
import os

Populating the interactive namespace from numpy and matplotlib


In [26]:
# Display python and library versions
!python --versions
print('Numpy ' + np.__version__)
print('TensorFlow ' + tf.__version__)
print('Keras ' + tf.keras.__version__)

unknown option --versions
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
Numpy 1.21.6
TensorFlow 2.8.2
Keras 2.8.0


# **Dataset Management**

In [27]:
# Loads the data and splits it into 60% training and 40% testing sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
y_test_old = y_test

x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

x_train = x_train[0:6000]
x_test = x_test[0:1000]
y_train = y_train[0:6000]
y_test = y_test[0:1000]

# **Clustering**

In [28]:
# Loads the base model for ANN tests and gets baseline accuracy for result comparison
base_model = tf.keras.models.load_model('drive/MyDrive/GE_practicum/CNN_base')
base_model.fit(x_train,y_train,epochs = 10,validation_data = (x_test,y_test), batch_size=32)
base_model.summary()

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(base_model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Epoch 1/10
188/188 [==============================] - 4s 19ms/step - loss: 0.0231 - accuracy: 0.9952 - val_loss: 0.1039 - val_accuracy: 0.9610
Epoch 2/10
188/188 [==============================] - 4s 23ms/step - loss: 0.0187 - accuracy: 0.9963 - val_loss: 0.1150 - val_accuracy: 0.9640
Epoch 3/10
188/188 [==============================] - 5s 24ms/step - loss: 0.0157 - accuracy: 0.9967 - val_loss: 0.1139 - val_accuracy: 0.9640
Epoch 4/10
188/188 [==============================] - 5s 26ms/step - loss: 0.0120 - accuracy: 0.9977 - val_loss: 0.1203 - val_accuracy: 0.9600
Epoch 5/10
188/188 [==============================] - 3s 14ms/step - loss: 0.0085 - accuracy: 0.9987 - val_loss: 0.1058 - val_accuracy: 0.9640
Epoch 6/10
188/188 [==============================] - 2s 13ms/step - loss: 0.0078 - accuracy: 0.9990 - val_loss: 0.1256 - val_accuracy: 0.9630
Epoch 7/10
188/188 [==============================] - 2s 13ms/step - loss: 0.0062 - accuracy: 0.9995 - val_loss: 0.1134 - val_accuracy: 0.9590

In [29]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

batch_size = 32
epochs = 10

num_images = x_train.shape[0]
end_step = np.ceil(num_images / batch_size).astype(np.float32) * epochs

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

pruned_model = prune_low_magnitude(base_model, **pruning_params)

opt = SGD(learning_rate=0.0001, momentum=0.9)
pruned_model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])
pruned_model.summary()


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d_  (None, 26, 26, 32)       610       
 13 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_max_poo  (None, 8, 8, 32)         1         
 ling2d_13 (PruneLowMagnitud                                     
 e)                                                              
                                                                 
 prune_low_magnitude_flatten  (None, 2048)             1         
 _13 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_dense_2  (None, 30)               122912    
 6 (PruneLowMagnitude)                                           
                                                     

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)


In [30]:
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
]

pruned_model.fit(x_train,y_train,epochs = epochs,validation_data = (x_test,y_test), batch_size=batch_size, callbacks=callbacks)
pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(pruned_model, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Epoch 1/10
188/188 [==============================] - 4s 14ms/step - loss: 0.0551 - accuracy: 0.9832 - val_loss: 0.1939 - val_accuracy: 0.9390
Epoch 2/10
188/188 [==============================] - 2s 13ms/step - loss: 0.0596 - accuracy: 0.9847 - val_loss: 0.1696 - val_accuracy: 0.9410
Epoch 3/10
188/188 [==============================] - 2s 13ms/step - loss: 0.1136 - accuracy: 0.9612 - val_loss: 0.2166 - val_accuracy: 0.9260
Epoch 4/10
188/188 [==============================] - 2s 13ms/step - loss: 0.1824 - accuracy: 0.9427 - val_loss: 0.3130 - val_accuracy: 0.9010
Epoch 5/10
188/188 [==============================] - 2s 13ms/step - loss: 0.2218 - accuracy: 0.9435 - val_loss: 0.3617 - val_accuracy: 0.8880
Epoch 6/10
188/188 [==============================] - 2s 13ms/step - loss: 0.2533 - accuracy: 0.9347 - val_loss: 0.3773 - val_accuracy: 0.8900
Epoch 7/10
188/188 [==============================] - 2s 13ms/step - loss: 0.2524 - accuracy: 0.9358 - val_loss: 0.3958 - val_accuracy: 0.8800

# **Quantization aware training**

In [31]:
quantize_model = tfmot.quantization.keras.quantize_model
model = quantize_model(pruned_model)

opt = SGD(learning_rate=0.0001, momentum=0.9)
model.compile(optimizer = opt,loss = 'categorical_crossentropy' , metrics=['accuracy'])

model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLay  (None, 28, 28, 1)        3         
 er)                                                             
                                                                 
 quant_conv2d_13 (QuantizeWr  (None, 26, 26, 32)       387       
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_13 (Qua  (None, 8, 8, 32)         1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_flatten_13 (QuantizeW  (None, 2048)             1         
 rapperV2)                                                       
                                                                 
 quant_dense_26 (QuantizeWra  (None, 30)             

In [32]:
# Performs quantization aware training
model.fit(x_train,y_train,epochs = 10,validation_data = (x_test,y_test), batch_size=32)

Epoch 1/10
188/188 [==============================] - 4s 19ms/step - loss: 0.2486 - accuracy: 0.9512 - val_loss: 0.3386 - val_accuracy: 0.9030
Epoch 2/10
188/188 [==============================] - 3s 18ms/step - loss: 0.2118 - accuracy: 0.9587 - val_loss: 0.3034 - val_accuracy: 0.9120
Epoch 3/10
188/188 [==============================] - 3s 18ms/step - loss: 0.1893 - accuracy: 0.9613 - val_loss: 0.2826 - val_accuracy: 0.9170
Epoch 4/10
188/188 [==============================] - 3s 18ms/step - loss: 0.1736 - accuracy: 0.9642 - val_loss: 0.2652 - val_accuracy: 0.9170
Epoch 5/10
188/188 [==============================] - 3s 18ms/step - loss: 0.1621 - accuracy: 0.9663 - val_loss: 0.2528 - val_accuracy: 0.9180
Epoch 6/10
188/188 [==============================] - 3s 18ms/step - loss: 0.1528 - accuracy: 0.9692 - val_loss: 0.2428 - val_accuracy: 0.9190
Epoch 7/10
188/188 [==============================] - 3s 18ms/step - loss: 0.1449 - accuracy: 0.9708 - val_loss: 0.2329 - val_accuracy: 0.9260

# **Quantization and conversion to tflite**

In [33]:
# Conversion to tflite

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

open('CNN_pruned.tflite', 'wb').write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp4nwnqzbw/assets


INFO:tensorflow:Assets written to: /tmp/tmp4nwnqzbw/assets
/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


66760

In [34]:
# Check tflite model characteristics
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)

=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the QUANTIZE op takes
tensor #0 as input and produces tensor #5 as output.

Subgraph#0 main(T#0) -> [T#15]
  Op#0 QUANTIZE(T#0) -> [T#5]
  Op#1 CONV_2D(T#5, T#6, T#2) -> [T#7]
  Op#2 MAX_POOL_2D(T#7) -> [T#8]
  Op#3 RESHAPE(T#8, T#1) -> [T#9]
  Op#4 FULLY_CONNECTED(T#9, T#10, T#3) -> [T#11]
  Op#5 FULLY_CONNECTED(T#11, T#12, T#4) -> [T#13]
  Op#6 SOFTMAX(T#13) -> [T#14]
  Op#7 DEQUANTIZE(T#14) -> [T#15]

Tensors of Subgraph#0
  T#0(serving_default_conv2d_13_input:0) shape_signature:[-1, 28, 28, 1], type:FLOAT32
  T#1(sequential_13/quant_flatten_13/Const) shape:[2], type:INT32 RO 8 bytes
  T#2(conv2d_13/bias) shape:[32], type:INT32 RO 128 bytes
  T#3(dense_26/bias) shape:[30], type:INT32 RO 120 bytes
  T#4(dense_27/bias) shape:[10], type:INT32 RO 40 bytes
  T#5(sequential_13/quantize_layer/AllValuesQuantize/FakeQuantWithMin

In [35]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
input_details = interpreter.get_input_details()

interpreter.allocate_tensors()
output_details = interpreter.get_output_details()

#Predictions from TFLite model
tfl_pred = []
tfl_pred_class = []
for i in range(len(x_test)):
    interpreter.set_tensor(input_details[0]["index"], x_test.astype('float32')[i:i+1,:])
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])
    tfl_pred.append(result)
    tfl_pred_class.append(argmax(result))

right_pred = [y_test_old[i] == tfl_pred_class[i] for i in range(len(y_test))]
acc = sum(right_pred)/len(right_pred)
print(acc)

0.933


# **Conversion to C array**

In [36]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [37]:
# Write TFLite model to a C source (or header) file
with open("CNN_pruned" + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, "CNN_pruned"))

# **Size Comparison**

In [38]:
def get_gzipped_model_size(file):
  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [39]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size('CNN_pruned.tflite')))

Size of gzipped baseline Keras model: 232908.00 bytes
Size of gzipped pruned Keras model: 74528.00 bytes
Size of gzipped pruned TFlite model: 25416.00 bytes
